# **Código original vídeo aula**

# 1. Criar a variavél de ambiente com a chave API Groq


In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os
project_root = Path.cwd().parent.parent  # caminho da raíz do projeto

# Carregar o .env
env_path = project_root / '.env'
load_dotenv(env_path)
api_key = os.getenv('GROQ_API_KEY')


# 2. Criar conexão com a API da Groq

In [ ]:
import os
from groq import Groq


client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[                                                              # Lista de mensagens da conversa
        {
            "role": "user",                                                 #indica que é mensagem do usuário
            "content": "Explain the importance of fast language models",    # o Texto da pergunta
        }
    ],
    model="llama-3.3-70b-versatile",                                        # Qual modelo utilizar
)

print(chat_completion.choices[0].message.content)

# choices[0]: primeira (e única) resposta gerada
# .message.content: extrai só o texto da resposta

# 3. Criar a Classe do Agente

In [ ]:
class Agent:
    def __init__(self, client, system): #Construtor, recebe os argumentos que serão os atributos do objeto
        self.client= client             #Conexão com API Groq,
        self.system= system             #Prompt que define o comportamento do agente | Define personalidade/função (ex: "Você é um assistente técnico")
        self.messages= []               #Armazenar o histórico da conversa
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=''): #Permite usar a classe como função
        if message:             
            self.messages.append({"role": "user", "content": message})  #Adiciona a pergunta (input) do usario,
        result= self.execute()                                          #Chamada da API do Groq
        self.messages.append({"role": "assistant", "content": result})  #Salva a resposta do modelo, resposta com base no histórico
        return result
    
    def execute(self):
        completion= client.chat.completions.create(
        messages=self.messages,                         #Envia todo o histórico
        model="llama-3.3-70b-versatile",                # Qual modelo está utilizando
    )
        return completion.choices[0].message.content

# 4. Implementando Padrão ReAct
  
    
    4.1. Thought   → Raciocínio sobre o problema

    4.2. Action    → Executa ferramenta

    4.3. PAUSE     → Aguarda resultado

    4.4. Observation → Recebe resultado da ferramenta
    
    4.5. (Volta ao Thought ou dá Answer final)
    


In [ ]:
#Prompt que define o comportamento do agente | Define personalidade/função (ex: "Você é um assistente técnico")
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [ ]:
#tools: Ferramenta de pesquisa, obter informações


def calculate(operation):
    return eval(operation)
def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":  
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.86e24
        case _:
            raise 0.0


# 5. Criar o agente e iterações manuais


In [ ]:
neil_tyson= Agent(client, system_prompt) #Cria um objeto/instância da classe Agente,

In [ ]:
result= neil_tyson("What is the mass of Earth times 5?")    #Input do usuário, usando a função __CALL__ e execute(), ambas definidas na classe Agent
print(result)

In [ ]:
neil_tyson.messages     #Mostrando a lista com o histórico de mensagens

In [ ]:
result=neil_tyson()     #Settando mensagem vazia ao agente, traz como o resultado o pensamento e ação.
print(result)

In [ ]:
observation= get_planet_mass("Earth")   #Obtendo via função o valor da massa da Terra disponível na função get_planet_mass()
print(observation)

In [ ]:
next_prompt= f"Observation: {observation}"  #Armazenando o valor da massa da terra obtido na célula anterior, ou seja, estamos fornecendo o dado obtido pela tool para o modelo.
result=neil_tyson(next_prompt)
print(result)                               #Resultado do modelo em passo da observação da tool e todo o histórico de mensagens.

In [ ]:
neil_tyson.messages     #Histórico construído de mensagens.

In [ ]:
result= neil_tyson(next_prompt) #Não entendi a necessidade deste bloco, a informação de next_prompt já foi informada ao agente.
print(result)

In [ ]:
observation= calculate("3.285e23 * 5")  #Tool/função que realiza a multiplicação, e então armazenar o valor da massa em Kg
print(observation)

In [ ]:
next_prompt= f"Observation: {observation}"  #Guardando no histórico de mensagens o valor da conta que foi realizado via tools, e então colocando na lista para o modelo pegar todo o contexto e reponder.
result=neil_tyson(next_prompt)
print(result)

In [ ]:
neil_tyson.messages #Histórico final das mensagens

# 6. Implementando looping automático

In [ ]:
import re   #ferramenta de Reg ex para normalização de estrura das strings

result= """
Action: calculate: 5.972e24 * 5
PAUSE
"""

action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)

In [ ]:
action

In [ ]:

import re

def agent_loop(max_iterations, system, query):  #max_iteration= limite de iterações, system= prompt de papel do modelo/agente, query= Demanda usuário
    agent = Agent(client, system)               #criando o agente (conexão com a API Groq, prompt de sistema)
    tools = ['calculate', 'get_planet_mass']    #lista das ferramentas, aqui temos apenas as duas funções, uma guarda as informações das massas dos planetas enquanto a outra calcula a massa.
    next_prompt = query                         #next_prompt= é a obtenção do input
    i = 0 #contador de iterações
    while i < max_iterations:
        i += 1
        print(f'ITERAÇÃO: {i}\n\n')
        result = agent(next_prompt)             #Obtém o output do modelo (pensamento + ferramentas) na primeira iteração é a própria query
        print(result)

        if "PAUSE" in result and "Action" in result:    #Estrutura de seleção para dados que passaram pelo modelo (Pause e Action são as flags)
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)  #normaliza o dado para saber qual é a ação / Ferramenta a ser usada
            chosen_tool = action[0][0] #Obtem qual é a função/ferramenta (calcular ou obter massa)
            arg = action[0][1]         #Obtém o argumento da função logo acima,

            if chosen_tool in tools:                            #executando a ferramenta sugerida pela LLM
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"     #output da ferramenta (massa da terra, por exemplo, ou valor da multiplicação) e será input na proxima iteração
            
            else:
                next_prompt = "Observation: tool not found"     #caso não tenha a ferramenta, há um log dizendo qhe não foi encontrada e enviada para a próxima iteração

            print(next_prompt)
            continue

        if "Answer" in result: #Será que se eu colocar a palavra "Answer" no input do usuário vai dar ruim??? acho q sim, 
            break
                

In [ ]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the Earth plus the mass of Mercury, and all of it times 5?") #resultado final das iterações